The purpose of this script is used to exercise the use of one hot encoding in categorical data analysis (here I did Linear Regression). I tried to use different approaches for this!

In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
import os

# set working directory
os.chdir(r"H:\Tutorial\Kaggle Competitions\AllState")
col_names = pd.read_csv('train.csv', nrows=1).columns
#print(list(col_names[0:20]))

# for exercise only
train = pd.read_csv('train.csv', nrows=60000,
                    usecols=list(col_names[0:10]) + list(col_names[65:]))
# for real run, read everything instead!
#train = pd.read_csv('train.csv')
print(train.shape)

train_cat_only = train.select_dtypes(include=[object])
train_num_only = train.select_dtypes(exclude=[object])
cat_cols = train_cat_only.columns.values
#print(train_cat_only.head(3))
#print(train_cat_only.tail(3))
#print(cat_cols)

# Get the categorical values into a 2D numpy array
#train_cat_values = np.array(train[cat_cols])
train_cat_only = np.array(train_cat_only)
#print(train_cat_only)

# convert categorical data to numerical. To do so, we first convert categorical
# data to int array and then convert int array to one hot encoding binary data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# for the first column for later stacking
le  = LabelEncoder()
train_cat_to_num = le.fit_transform(train_cat_only[:,0])

for idx in range(1, len(cat_cols)):
    le = LabelEncoder()
    train_cat_to_num = np.column_stack((train_cat_to_num, 
                                        le.fit_transform(train_cat_only[:,idx])))

train_cat_to_num = train_cat_to_num.astype(np.float32)
#print(train_cat_to_num[0:2,])

# for one-hot coding
ohe = OneHotEncoder()
train_cat_to_num = ohe.fit_transform(train_cat_to_num)
#print(ohe.n_values_)
#print(ohe.feature_indices_)

# now we need to create the expanded column headers to form a data.frame from np.array
col_names = []
for i in range(0, len(cat_cols)):
    for j in range(0, ohe.n_values_[i]):
        col_names.append(cat_cols[i] + '_' + str(j))        
#print(col_names)
train_cat_to_num_df = pd.DataFrame(train_cat_to_num.toarray(), columns=col_names)
#print(train_cat_to_num_df.head(2))

# now need to combine with the original numerical data
# but we have to get the target 'loss' out, before combining
target = train_num_only['loss']
print(target.head())
train_num_only.drop(['loss'], 1, inplace=True)
train_num_only[col_names] = train_cat_to_num_df[col_names]
print(train_num_only.head(3))

# now do linear Regression
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing, cross_validation
from sklearn.metrics import mean_absolute_error

X = train_num_only
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, target, test_size=0.2)

clf = LinearRegression()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(y_pred.shape)
print(y_test.shape)
mae = mean_absolute_error(np.array(y_test), np.array(y_pred), multioutput='raw_values')
#accuracy = clf.score(X_test, y_test)
print("The accuracy is:", mae)

(60000, 77)
0    2213.18
1    1283.60
2    3005.09
3     939.85
4    2763.85
Name: loss, dtype: float64


C:\Users\ting\Anaconda3\lib\site-packages\ipykernel\__main__.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\ting\Anaconda3\lib\site-packages\pandas\core\frame.py:2369: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


   id     cont1     cont2     cont3     cont4     cont5     cont6     cont7  \
0   1  0.726300  0.245921  0.187583  0.789639  0.310061  0.718367  0.335060   
1   2  0.330514  0.737068  0.592681  0.614134  0.885834  0.438917  0.436585   
2   5  0.261841  0.358319  0.484196  0.236924  0.397069  0.289648  0.315545   

     cont8    cont9     ...      cat116_269  cat116_270  cat116_271  \
0  0.30260  0.67135     ...             0.0         0.0         0.0   
1  0.60087  0.35127     ...             0.0         0.0         0.0   
2  0.27320  0.26076     ...             0.0         0.0         0.0   

   cat116_272  cat116_273  cat116_274  cat116_275  cat116_276  cat116_277  \
0         0.0         0.0         0.0         0.0         0.0         0.0   
1         0.0         0.0         0.0         0.0         0.0         0.0   
2         0.0         0.0         0.0         0.0         0.0         0.0   

   cat116_278  
0         0.0  
1         0.0  
2         0.0  

[3 rows x 960 columns]
(

The previous method has to manually add the column names, which is not quite easy for some applications. Here I use a different approach!

In [41]:
import pandas as pd
import numpy as py
import os

# set up working directory
os.chdir(r"H:\Tutorial\Kaggle Competitions\AllState")

# read training dataset
# if you read everything for final run, use
#train = pd.read_csv('train.csv')

# for exercise, use the following
col_names = pd.read_csv('train.csv', nrows=1).columns
train = pd.read_csv("train.csv", nrows=6000, 
                    usecols=list(col_names[0:10]) + list(col_names[65:]))
print(train.shape)

# get all categorical data and all numerical data separately
train_cat_only = train.select_dtypes(include=[object])
train_num_only = train.select_dtypes(exclude=[object])
print(train_cat_only.shape)
print(train_num_only.shape)

# get the target column from train_num_only and remove it from train_num_only after that
target = train_num_only['loss']
train_num_only.drop(['loss'], 1, inplace=True)
print(target.head(3))

# convert categorical data to one hot encoded array

# Using DictVectorizer() will loose the column names, even though you could get it
# back using feature_names_. The order is inline with the final results
#train_cat_only_sorted = train_cat_only.reindex_axis(sorted(train_cat_only.columns), axis=1)
#print(train_cat_only_sorted.head(3))
#print(train_cat_only.head(3))

from sklearn.feature_extraction import DictVectorizer
dv = DictVectorizer(sort=False)
train_cat_to_num = dv.fit_transform(train_cat_only_sorted.to_dict("records")).toarray()
print(train_cat_to_num.shape)
#print(dv.feature_names_)

# to order the column names if you use vocabulary_
#import operator
#voc = dv.vocabulary_
#ordered_col_names = sorted(voc.items(), key=operator.itemgetter(1))
#print(ordered_col_names)

# as the feature_names_ contains the column names corresponding to the train_cat_to_num
# we can re-build the dataframe for further anaysis
train_cat_to_num_df = pd.DataFrame(train_cat_to_num, columns=list(dv.feature_names_))
print(train_cat_to_num_df.head(3))

# use pandas get_dummies() is the best way to do it and get all column names 
train_cat_to_num = pd.get_dummies(train_cat_only)
cat_names = train_cat_to_num.columns.values
print(train_cat_to_num.shape)
#print(train_cat_to_num.head(3))
#print(cat_names)

# now combine both categorical data and numeric data
X = train_num_only.copy()
X[cat_names] = train_cat_to_num[cat_names]
print(X.shape)

# preprocessing and validation
from sklearn import preprocessing, cross_validation
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

X = preprocessing.scale(X)
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, target, test_size=0.2)

clf = LinearRegression()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
mae = mean_absolute_error(y_pred, y_test,multioutput='raw_values')
print("Mean Absolute Error is: ", mae)

(6000, 77)
(6000, 61)
(6000, 16)
0    2213.18
1    1283.60
2    3005.09
Name: loss, dtype: float64


C:\Users\ting\Anaconda3\lib\site-packages\ipykernel\__main__.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(6000, 744)
   cat73=A  cat111=C  cat74=A  cat6=A  cat66=A  cat69=A  cat3=A  cat84=C  \
0      1.0       1.0      1.0     1.0      1.0      1.0     1.0      1.0   
1      1.0       0.0      1.0     1.0      1.0      1.0     1.0      1.0   
2      1.0       0.0      1.0     1.0      1.0      1.0     1.0      1.0   

   cat104=I  cat102=A    ...      cat116=AW  cat110=P  cat110=DO  cat116=KY  \
0       1.0       1.0    ...            0.0       0.0        0.0        0.0   
1       0.0       1.0    ...            0.0       0.0        0.0        0.0   
2       0.0       1.0    ...            0.0       0.0        0.0        0.0   

   cat109=AV  cat110=CT  cat114=V  cat116=CA  cat116=MK  cat110=AJ  
0        0.0        0.0       0.0        0.0        0.0        0.0  
1        0.0        0.0       0.0        0.0        0.0        0.0  
2        0.0        0.0       0.0        0.0        0.0        0.0  

[3 rows x 744 columns]
(6000, 744)
(6000, 759)
Mean Absolute Error is:  [  6.74967153e+15